# **DATA PREPROCESSING BASED ON EDA INSIGHTS**

This notebook implements preprocessing steps based on the comprehensive EDA findings and recommendations. We'll follow the evidence-based approach from the EDA report to ensure our preprocessing aligns with the data patterns discovered.
Based on the EDA report, we will:

1. **Handle Skewed Variables** - Log-transform `CoapplicantIncome`,  `ApplicantIncome `

2. **Feature Engineering**:

Feature Engineering Suggestions:
- Total_Income = ApplicantIncome + CoapplicantIncome
- EMI = LoanAmount / Loan_Amount_Term
- Income_to_Loan_Ratio = Total_Income / LoanAmount

3. **Feature Selection** - Keep high-signal features
4. **Scaling** - StandardScaler for distance-based models


 Final Preprocessing Pipeline (Suggested Order)
1. Handle missing values (mode/median).

2. Clip outliers (IQR)(optional).

3. Apply log transformations on skewed variables.

4. Feature engineering (TotalIncome, EMI, Ratios).

5. Encode categorical variables (Label + One-Hot).

6. Scale numerical features (StandardScaler/RobustScaler).

7. Address class imbalance (SMOTE or class weights).

8. Perform feature selection 

9. Split data into train/test sets (e.g., 80/20).

#### **1. Import Libraries and Load Data**

In [57]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import warnings
warnings.filterwarnings('ignore')

# Preprocessing libraries
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix

# Statistical libraries
from scipy import stats
from scipy.stats import zscore, skew

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [58]:
# load dataset through url
train_url = "https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/home_loan_train.csv"

test_url = "https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/home_loan_test.csv"


df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
print("Dataset loaded sucessfuly")

Dataset loaded sucessfuly


In [59]:
df_train.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

#### **2. EDA-Based Data Quality Assessment**

In [60]:
# Create a copy for preprocessing
df_processed = df_train.copy()
df_processed.set_index("Loan_ID", inplace=True)
# 1. Check for missing values (EDA showed no missing values)
print("\n1. Missing Values:")
missing_values = df_processed.isnull().sum()
if missing_values.sum() > 0:
    print(missing_values[missing_values > 0])
else:
    print("No missing values found")

# 2. Check for duplicates
print("\n2. Duplicate Rows:")
duplicates = df_processed.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
if duplicates > 0:
    print(f"Percentage of duplicates: {(duplicates/len(df_processed))*100:.2f}%")

#Loan_Amount_Term and Credit_history are more categorical than continous
df_processed["Credit_History"] = df_processed["Credit_History"].astype(str)
df_processed["Loan_Amount_Term"] = df_processed["Loan_Amount_Term"] .astype(str)
# re-map Credit_history
df_processed["Credit_History"] = df_processed["Credit_History"].map({"1.0":"good", "0.0":"bad" })

# 3. Identify number of outliers
print("\n2. Outlier Rows:")
df_processed['Loan_Status'] = df_processed['Loan_Status'].map({'Y': 1, 'N':0})
num_cols = df_processed.select_dtypes(include=["float64","int64"]).columns
df_processed[num_cols].describe()
outlier_columns = []
for i in num_cols:
    q1 = df_processed[i].quantile(0.25)
    q3 = df_processed[i].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outlier_count = ((df_processed[i] < lower) | (df_processed[i] > upper)).sum()
    print(f"The number of outliers for {i} is {outlier_count}")
    if outlier_count > 0:
        outlier_columns.append(i)

# 3. Check skewness for variables identified in EDA as right-skewed
print("\n3. Skewness Analysis (EDA identified right-skewed variables):")
skewed_vars = ['ApplicantIncome', 'CoapplicantIncome']
for var in skewed_vars:
    if var in df_processed.columns:
        skewness = skew(df_processed[var])
        print(f"{var}: skewness = {skewness:.3f} ({'right-skewed' if skewness > 0.5 else 'approximately normal'})")

# 4. select High Signal features based EDA
# let make our target value a numerical column
print("\nHigh-signal features (|correlation with | > 0.011):")
corrs = df_processed[num_cols].corr()
print(corrs)



1. Missing Values:
Gender              13
Married              3
Dependents          15
Self_Employed       32
LoanAmount          22
Loan_Amount_Term    14
Credit_History      50
dtype: int64

2. Duplicate Rows:
Number of duplicate rows: 0

2. Outlier Rows:
The number of outliers for ApplicantIncome is 50
The number of outliers for CoapplicantIncome is 18
The number of outliers for LoanAmount is 39
The number of outliers for Loan_Status is 0

3. Skewness Analysis (EDA identified right-skewed variables):
ApplicantIncome: skewness = 6.524 (right-skewed)
CoapplicantIncome: skewness = 7.473 (right-skewed)

High-signal features (|correlation with | > 0.011):
                   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Status
ApplicantIncome           1.000000          -0.116605    0.570909    -0.004710
CoapplicantIncome        -0.116605           1.000000    0.188619    -0.059187
LoanAmount                0.570909           0.188619    1.000000    -0.037318
Loan_Status         

#### **1. Handle Missing Values**


In [61]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, LP001002 to LP002990
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             601 non-null    object 
 1   Married            611 non-null    object 
 2   Dependents         599 non-null    object 
 3   Education          614 non-null    object 
 4   Self_Employed      582 non-null    object 
 5   ApplicantIncome    614 non-null    int64  
 6   CoapplicantIncome  614 non-null    float64
 7   LoanAmount         592 non-null    float64
 8   Loan_Amount_Term   614 non-null    object 
 9   Credit_History     564 non-null    object 
 10  Property_Area      614 non-null    object 
 11  Loan_Status        614 non-null    int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 78.5+ KB


In [ ]:
# handle missing values in numerical columns
 # extracts numerical columns
num_cols = df_processed.select_dtypes(include=["float64","int64"]).columns 
#Loan_Amount_Term and Credit_history are more categorical than continous


# check for missing values
df_processed[num_cols].isna().sum()

# fill the  missing values in numerical columns with median
df_processed.loc[:, num_cols] = df_processed[num_cols].fillna(df_processed[num_cols].median())  

# # recheck for missing values
df_processed[num_cols].isna().sum()

ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
dtype: int64

In [ ]:
# # handle missing values in categorical columns
# # extracts categorical column
cat_cols = df_processed.select_dtypes(include="object").columns 
#Loan_Amount_Term and Credit_history are more categorical than continous

# check for missing values
df_processed[cat_cols].isna().sum()

# get the mode of the columns
modes = df_processed[cat_cols].apply(lambda x: x.value_counts().index[0])

# fill the missing values in the catgorical columns with modes
df_processed[cat_cols] = df_train[cat_cols].fillna(modes)


# rechecck the missing values
df_processed[cat_cols].isna().sum()

Loan_ID          0
Gender           0
Married          0
Dependents       0
Education        0
Self_Employed    0
Property_Area    0
Loan_Status      0
dtype: int64

#### **2. Outlier Treatment (EDA Recommendation)**

In [64]:
#Locate outliers and handle using IQR method and clip
def clean_outliers(data, i):
    q1 = data[i].quantile(0.25)
    q3 = data[i].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    data.loc[:, i] = np.where((data.loc[:, i] < lower) | (data.loc[:, i] > upper), 
                       data[i].clip(lower = lower, upper = upper).astype(data[i].dtype), data[i])
    return data

for i in outlier_columns:
    df_processed = clean_outliers(df_train, i)

df_processed.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


#### **3. Log-Transform Skewed Variables (EDA Recommendation)**

In [65]:
# Log-transform skewed variables as recommended by EDA
print("=== LOG-TRANSFORMING SKEWED VARIABLES ===")
print("EDA identified these variables as right-skewed and recommended log transformation:")
# Variables to log-transform based on EDA findings
skewed_vars = ['ApplicantIncome', 'CoapplicantIncome']
for var in skewed_vars:
    if var in df_processed.columns:
        # Check if variable has zero or negative values
        min_val = df_processed[var].min()
        print(min_val)
        if min_val <= 0.0:
            # Use log1p for variables with zeros
            df_processed[f'{var}_log'] = np.log1p(df_processed[var])
            print(f"✓ {var}: Applied log1p transformation (had {min_val:.3f} minimum value)")
        else:
            # Use log for positive values only
            df_processed[f'{var}_log'] = np.log(df_processed[var])
            print(f"✓ {var}: Applied log transformation")
        
        # Check skewness before and after
        original_skew = skew(df_processed[var])
        transformed_skew = skew(df_processed[f'{var}_log'])
        print(f"  Original skewness: {original_skew:.3f} → Transformed skewness: {transformed_skew:.3f}")
   

=== LOG-TRANSFORMING SKEWED VARIABLES ===
EDA identified these variables as right-skewed and recommended log transformation:
150
✓ ApplicantIncome: Applied log transformation
  Original skewness: 1.037 → Transformed skewness: -0.594
0.0
✓ CoapplicantIncome: Applied log1p transformation (had 0.000 minimum value)
  Original skewness: 1.010 → Transformed skewness: -0.184


#### **4. Feature Engineering**

Feature Engineering Suggestions:
- Total_Income = ApplicantIncome + CoapplicantIncome
- EMI = LoanAmount / Loan_Amount_Term
- Income_to_Loan_Ratio = Total_Income / LoanAmount

In [66]:
# change the dtype
df_processed["Loan_Amount_Term"] = df_train["Loan_Amount_Term"] .astype(float)

In [67]:
df_processed.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,8.674026,0.000000
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,8.430109,7.319202
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,8.006368,0.000000
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,7.856707,7.765993
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,8.699515,0.000000


In [68]:
#creating column for total income  = ApplicantIncome + CoapplicantIncome
df_processed["Total_income"] = df_processed["ApplicantIncome"] + df_processed["CoapplicantIncome"]
df_processed.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log,Total_income
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,8.674026,0.000000,5849.0
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,8.430109,7.319202,6091.0
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,8.006368,0.000000,3000.0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,7.856707,7.765993,4941.0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,8.699515,0.000000,6000.0


In [69]:
#creating column for Equated Monthly Installment(EMI) = LoanAmount / Loan_Amount_Term
df_processed["EMI"] = df_processed["LoanAmount"] / df_processed["Loan_Amount_Term"]
df_processed.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log,Total_income,EMI
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,8.674026,0.000000,5849.0,0.355556
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,8.430109,7.319202,6091.0,0.355556
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,8.006368,0.000000,3000.0,0.183333
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,7.856707,7.765993,4941.0,0.333333
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,8.699515,0.000000,6000.0,0.391667


In [70]:
# - Income_to_Loan_Ratio = Total_Income / LoanAmount
df_processed["Income_to_Loan_Ratio"] = df_processed["Total_income"] / df_processed["LoanAmount"]
df_processed.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log,Total_income,EMI,Income_to_Loan_Ratio
0,LP001002,Male,No,0,Graduate,No,5849,0.0,128.0,360.0,1.0,Urban,Y,8.674026,0.000000,5849.0,0.355556,45.695312
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,8.430109,7.319202,6091.0,0.355556,47.585938
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,8.006368,0.000000,3000.0,0.183333,45.454545
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,7.856707,7.765993,4941.0,0.333333,41.175000
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,8.699515,0.000000,6000.0,0.391667,42.553191


 #### **5.Encode categorical variables (Label + One-Hot)**

In [71]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Loan_ID                614 non-null    object 
 1   Gender                 614 non-null    object 
 2   Married                614 non-null    object 
 3   Dependents             614 non-null    object 
 4   Education              614 non-null    object 
 5   Self_Employed          614 non-null    object 
 6   ApplicantIncome        614 non-null    int64  
 7   CoapplicantIncome      614 non-null    float64
 8   LoanAmount             614 non-null    float64
 9   Loan_Amount_Term       614 non-null    float64
 10  Credit_History         614 non-null    float64
 11  Property_Area          614 non-null    object 
 12  Loan_Status            614 non-null    object 
 13  ApplicantIncome_log    614 non-null    float64
 14  CoapplicantIncome_log  614 non-null    float64
 15  Total_

In [56]:
df_processed["Credit_History"]

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
609    1.0
610    1.0
611    1.0
612    1.0
613    0.0
Name: Credit_History, Length: 614, dtype: float64

In [ ]:
columns_to_encode = ["Gender","Married","Dependents","Education","Self_Employed",]